<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_03_5_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 3: Large Language Models**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 3 Material

* Part 3.1: Foundation Models [[Video]](https://www.youtube.com/watch?v=Gb0tk5qq1fA) [[Notebook]](t81_559_class_03_1_llm.ipynb)
* Part 3.2: Text Generation [[Video]](https://www.youtube.com/watch?v=lB97Lqt7q58) [[Notebook]](t81_559_class_03_2_text_gen.ipynb)
* Part 3.3: Text Summarization [[Video]](https://www.youtube.com/watch?v=3MoIUXE2eEU) [[Notebook]](t81_559_class_03_3_text_summary.ipynb)
* Part 3.4: Text Classification [[Video]](https://www.youtube.com/watch?v=2VpOwFIGmA8) [[Notebook]](t81_559_class_03_4_classification.ipynb)
* **Part 3.5: LLM Writes a Book** [[Video]](https://www.youtube.com/watch?v=iU40Rttlb_Q) [[Notebook]](t81_559_class_03_5_book.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [1]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai langchain_community pypdf pdfkit
    !apt-get install wkhtmltopdf

Note: using Google CoLab
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy
  libavahi-core7 libavahi-glib1 libdaemon0 libevdev2 libgudev-1.0-0 libhyphen0
  libinput-bin libinput10 libjson-glib-1.0-0 libjson-glib-1.0-common
  libmbim-glib4 libmbim-proxy libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200
  libnotify4 libnss-mdns libproxy1v5 libqmi-glib5 libqmi-proxy libqt5co

# 3.5: LLM Writes a Book

This section will demonstrate how to effectively utilize the LLM to write a book. Writing a book is a uniquely complex challenge that requires planning and execution. Unlike simpler tasks, drafting a full-length book is a journey that extends beyond a single session with an LLM due to its extensive length and depth. However, with a systematic approach, you can harness the capabilities of an LLM to create a comprehensive manuscript, empowering you to conquer this challenge.

This section presents a method to craft a book using an LLM, which involves breaking down the book creation process into manageable segments. Our journey begins with a foundational step: selecting a subject for the book. Once a subject is chosen, the process unfolds through a series of iterative interactions with the LLM, each building upon the last to gradually construct the complete book, igniting your passion for writing.

Initially, we prompt the LLM to generate a random title based on the given subject. This title sets the thematic tone for the book and guides the subsequent steps. Following the title, we request the LLM to create a random synopsis. This synopsis acts as a narrative backbone, offering a brief overview of the book’s content and direction.

With a synopsis, the next step is to generate a table of contents. This table outlines the main chapters and sections of the book, providing a structured framework that organizes the content logically and cohesively. It is from this framework that the book starts to take shape.
The next phase involves creating each chapter. We engage in a focused session with the LLM for every chapter outlined in the table of contents. During each session, we provide the LLM with the synopsis and the complete table of contents to ensure consistency and continuity throughout the book. This iterative process allows each chapter to be crafted with attention to detail, ensuring it aligns with the previously established narrative and thematic elements.

By methodically iterating through these steps, we harness the capabilities of an LLM to transform a simple idea into a detailed and well-structured book. This chapter will guide you through each stage, providing practical advice on collaborating with an LLM to achieve more complex tasks.

We begin by accessing a large language model with a temperature of 0.7. We use a higher temperature to encourage creativity.

In [2]:
from langchain.chains.summarize import load_summarize_chain
from langchain import OpenAI, PromptTemplate
from langchain_openai import ChatOpenAI
from IPython.display import display_markdown

MODEL = 'gpt-5-mini'

llm = ChatOpenAI(
        model=MODEL,
        temperature=0.7,
        n=1
    )


We create a simple utility function to query the LLM with a custom system prompt. The system prompt tells the LLM that it is assisting in creating a book.

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

def query_llm(prompt):
  messages = [
      SystemMessage(
          content="You are assistant helping to write a book."
      ),
      HumanMessage(
          content=prompt
      ),
  ]

  output = llm.invoke(messages)
  return output.content


## Generate Title, Synopsis and Table of Contents

For this book, we allow the user to specify the subject specified by the SUBJECT variable. We then request the LLM to generate a random title based on this subject. It is essential that the prompt request that the LLM only the; LLMs often like to prefix with text such as "Here is a random title."

In [4]:
SUBJECT = "international spy thriller"

title = query_llm(f"""
Give me a random title for a book on the subject '{SUBJECT}'.
Return only the title, no additional text.
""").strip(" '\"")
print(title)

The Atlas Directive


Now that we have a title, we can request a random synopsis of the book.

In [5]:
synopsis = query_llm(f"""
Give me a synopsis for a book of the title '{SUBJECT}' for a book on the subject '{SUBJECT}'.
Return only the synopsis, no additional text.
""").strip(" '\"")
print(synopsis)

When a routine data sweep in Prague turns up a fragment of a dead diplomat’s encrypted journal, burned into a memory chip and smuggled inside a violin case, ex-MI6 operative Jonah Hale is pulled back into the shadows he swore to leave. The chip hints at Project Meridian — a covert network of influence operations, black budgets and clandestine alliances stretching from the corridors of power in London and Washington to the back alleys of Istanbul and the desert strongholds of North Africa. As Jonah traces the trail, a global game of cat and mouse ignites: MI6, the CIA, Russia’s SVR and a private security cartel all claim pieces of the truth, and none can be trusted. 

Partnering with an enigmatic Chinese cyber-analyst, Lin Wei, and a disillusioned Mossad field officer, Ruth Ben-Ami, Jonah must decode a conspiracy that uses modern surveillance, old-school espionage and a roster of double agents to reshape geopolitical reality. Targets fall, allies turn on each other, and a lover’s betray

Next, we generate the table of contents. For this generation, we provide all previous information. We also request a particular format for the table of contents. You may notice that I ask for the chapter numbers, even though they are an increasing numeric count and could easily be derived. This process works better because the LLM wants to provide the chapter numbers, and attempts to suppress chapter numbers are often inconsistent. It is easier to allow the LLM to generate chapter numbers but control where it generates them so that I can consistently remove them later.

In [6]:
toc = query_llm(f"""
Give me a table of contents for a book of the title '{title}' for a book on
the subject '{SUBJECT}' the book synopsis is '{synopsis}'.
Return the table of contents as a list of chapter titles.
Separate the chapter number and chapter title with a pipe character '|'.
Return only the chapter names, no additional text.
Do not use more than 12 chapters.
Do not number the chapters.
""").strip(" '\"")
print(toc)

| The Violin Case
| Echoes in Prague
| Return to the Shadows
| Meridian's Thread
| Lin Wei
| Ruth Ben‑Ami
| Istanbul Nights
| Black Budgets
| The Lover's Betrayal
| Desert Strongholds
| Murmansk Ice
| The Suppressed Truth


We must now parse the table of contents and remove the pipes and chapter numbers.

In [7]:
# Split the string into lines
lines = toc.splitlines()

# Extract titles using list comprehension
toc2 = [line.split('|')[1].strip() for line in lines if line]

# Print the list of titles
print(toc2)

['The Violin Case', 'Echoes in Prague', 'Return to the Shadows', "Meridian's Thread", 'Lin Wei', 'Ruth Ben‑Ami', 'Istanbul Nights', 'Black Budgets', "The Lover's Betrayal", 'Desert Strongholds', 'Murmansk Ice', 'The Suppressed Truth']


## Generate the Chapters of the Book

Next, we create a function capable of producing the text that makes up a chapter. To ensure that the function has enough context to generate each chapter, we provide the synopsis, the table of contents, and the chapter number. To test this code, we request that it develop a single chapter.

In [8]:
def render_chapter(num, chapter_title, title, subject, synopsis, toc):
  txt = query_llm(f"""
  Write Chapter {num}, titled "{chapter_title}" for a book of the title '{title}' for a book on
  the subject '{subject}' the book synopsis is '{synopsis}' the table of contents is '{toc}'.
  Give me only the chapter text, no chapter heading, no chapter title, number, no additional text.
  """).strip(" '\"")
  return txt

txt = render_chapter(1, toc2[0], title, SUBJECT, synopsis, toc)
print(txt)

The case smelled of rosin and varnish and the stale apology of half-smoked cigarettes — the scent of a life that had traveled more than it had ever wanted to stay. It was a cheap, battered instrument case, the molded interior frayed where a violin’s shoulder had worn the foam to a shine. Someone had slit the lining along the seam and tucked a sliver of metal into the hollow like contraband. Then they’d closed it and carried it through a city that still remembered empires.

Jonah Hale held it in the fluorescent light of the Prague municipal lab and thought about all the clever ways the world hid dangerous things in plain sight. He had learned to see the logic of hiding — who would suspect a violin, who would think to look inside the soft cradle of an instrument for anything other than a violin? It was elegant. It was stupid. And like most elegant, stupid things, it was effective.

“You can say the word,” said the woman across the bench. She had the quick, flat accent of a forensic analy

We can now generate the entire book in Markdown, which allows some formatting. We begin by rendering the title and synopsis, the table of contents, and each chapter.

In [9]:
book = ""

# Render the title and synopsis
book += f"# {title}\n"
book += f"{synopsis}\n"

# Render the toc
book += f"\n## Table of Contents\n\n"
num = 1
for chapter_title in toc2:
  book += f"{num}. {chapter_title}\n"
  num += 1

# Render the book
chapter = 1
for chapter_title in toc2:
  print(f"Rendering chapter {chapter}/{len(toc2)}: {chapter_title}")
  txt = render_chapter(chapter, chapter_title, title, SUBJECT, synopsis, toc)
  book += f"\n\n## Chapter {chapter}: {chapter_title}\n"
  book += f"{txt}\n"
  chapter += 1

Rendering chapter 1/12: The Violin Case
Rendering chapter 2/12: Echoes in Prague
Rendering chapter 3/12: Return to the Shadows
Rendering chapter 4/12: Meridian's Thread
Rendering chapter 5/12: Lin Wei
Rendering chapter 6/12: Ruth Ben‑Ami
Rendering chapter 7/12: Istanbul Nights
Rendering chapter 8/12: Black Budgets
Rendering chapter 9/12: The Lover's Betrayal
Rendering chapter 10/12: Desert Strongholds
Rendering chapter 11/12: Murmansk Ice
Rendering chapter 12/12: The Suppressed Truth


## Generate a PDF of the Book

Now that we have generated the book, we have saved it as a PDF.

In [10]:
import markdown
import pdfkit

# Convert Markdown to HTML
html = markdown.markdown(book)
options = {
    'encoding': "UTF-8",  # Ensures UTF-8 encoding
}

pdfkit.from_string(html, 'output.pdf', options=options)

True

We can now download the generated book.

In [11]:
from google.colab import files
files.download("output.pdf")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>